# Predicitng Stock/Weather prices using neural networks

## import relevant libraries 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from keras.models import Sequential
import matplotlib.patches as mpatches
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import os


## Load data from the file and visualize the data

In [ ]:
df = pd.read_csv('testset.csv') # Loading the data from file
df.head()

# convert datetime columns from object type to datatime64 and set it as index

In [ ]:
df['datetime_utc'] = pd.to_datetime(df['datetime_utc'])
df.set_index('datetime_utc', inplace= True)

# Resample the colums as per Day/Hour/Mins as per requirements

In [ ]:
df =df.resample('H').mean() #The index is based on hours and mean value of the data from that specific hour is filled in the columns

# Select the columns you want to predict

In [ ]:
df = df['Open' ]

# Fill the empty slots 

In [ ]:
df = df.ffill().bfill()
df.mean()# we will fill the null row

# Plot the data 

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(df)
plt.title('Time Series')
plt.xlabel('Date')
plt.ylabel('Stock')
plt.show()

# Convert the data to float and reshape to 2D

In [ ]:
df=df.values
df = df.astype('float32')

In [ ]:
df=df.reshape(df.shape[0],1)
df

#  Transform the data using minmax scalr iwth the given range

In [ ]:
scaler= MinMaxScaler(feature_range=(0,1))
sc = scaler.fit_transform(df)

# Create Xtrain and Ytrain based on your requirements

In [ ]:
timestep = 10 #Steps used to train before predicting the next point
X=[]
Y=[]
for i in range(1,len(sc)- (timestep)):
    X.append(sc[i:i+timestep])
    Y.append(sc[i+timestep])
X=np.asanyarray(X)
Y=np.asanyarray(Y)
length = len(sc)-100

# shape of the input data varies form model to model
Xtrain = X[:length,:,:] 
Xtest = X[length:,:,:]    
Ytrain = Y[:length]    
Ytest= Y[k-1:-1]   

# Import the libraries

In [ ]:
from keras.layers import Dense,RepeatVector
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D


# Build the model Bidirectional

Below model is keras Bideirectional model

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(100, activation='sigmoid'),input_shape=(timestep,1)))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(Xtrain,Ytrain,epochs=10, verbose=0 )
model.summary()

# save model

In [ ]:
model.save('model.h5')

# load model

In [ ]:
model = load_model('model.h5')

# Predict the results and inverse transform them

In [ ]:
preds = model.predict(Xtest)
preds = scaler.inverse_transform(preds)



# Inverse transfrom test data to compare with the predicted results

In [ ]:
Ytest=np.asanyarray(Ytest)  
Ytest=Ytest.reshape(-1,1) 
Ytest = scaler.inverse_transform(Ytest)

# Mean squaled error is calculated to measure the accuracy of precdiction

Measuring accuracy varies based on the output and requirements

In [ ]:
mean_squared_error(Ytest,preds) 

# Plot the predicted and True results to visualize

In [ ]:
plt.figure(figsize=(20,9))
plt.plot(Ytest)
plt.plot(preds_cnn1)
plt.legend(('Test','Predicted'))
plt.show()